In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from dotenv import load_dotenv
import os

# Load variables from .env into environment
load_dotenv()

# Initialize model with API key and base URL from .env
model = ChatOpenAI(
    model="text-embedding-3-small",
    api_key=os.getenv("OPEN_API_KEY"),        # match variable name in .env
    openai_api_base=os.getenv("OPENAI_BASE_URL")
)

In [2]:
# !pip install -q langchain-openai langchain-core requests

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [4]:
# tool create

@tool
def multiply(a: int, b: int) -> int:
  """Given 2 numbers a and b this tool returns their product"""
  return a * b

In [5]:
print(multiply.invoke({'a':3, 'b':4}))

12


In [6]:
multiply.name

'multiply'

In [7]:
multiply.description

'Given 2 numbers a and b this tool returns their product'

In [8]:
multiply.args

{'a': {'title': 'A', 'type': 'integer'},
 'b': {'title': 'B', 'type': 'integer'}}

In [9]:
# tool binding

In [10]:
llm =  ChatOpenAI(
    model="gpt-4o-mini",
    api_key=os.getenv("OPEN_API_KEY"),        # match variable name in .env
    openai_api_base=os.getenv("OPENAI_BASE_URL")
)

In [11]:
llm_with_tools=llm.bind_tools([multiply])

In [12]:
# llm.invoke('hi')

In [13]:
llm_with_tools = llm.bind_tools([multiply])

In [14]:
llm_with_tools.invoke('Hi how are you')

AIMessage(content="I'm just a program, so I don't have feelings, but I'm here and ready to help you! How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 57, 'total_tokens': 85, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_8bda4d3a2c', 'id': 'chatcmpl-CDTCog1JfTdng8K299RDLWr5hbjzX', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--60abd9b7-dbcb-4fa2-9789-0ce8e13f0108-0', usage_metadata={'input_tokens': 57, 'output_tokens': 28, 'total_tokens': 85, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [15]:
query = HumanMessage('can you multiply 3 with 1000')

In [16]:
messages = [query]

In [17]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={})]

In [18]:
result = llm_with_tools.invoke(messages)

In [19]:
messages.append(result)

In [20]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_oH5fpEySIA6sF44mywhcWINL', 'function': {'arguments': '{"a":3,"b":1000}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 62, 'total_tokens': 80, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_8bda4d3a2c', 'id': 'chatcmpl-CDTCpwYo7TSYSlhyUYLPZAX8HuMcC', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--4467a30d-acce-45e8-9bb5-55858ab7d724-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 1000}, 'id': 'call_oH5fpEySIA6sF44mywhcWINL', 'type': 'tool_call'}], usage_metada

In [21]:
tool_result = multiply.invoke(result.tool_calls[0])

In [22]:
tool_result

ToolMessage(content='3000', name='multiply', tool_call_id='call_oH5fpEySIA6sF44mywhcWINL')

In [23]:
messages.append(tool_result)

In [24]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_oH5fpEySIA6sF44mywhcWINL', 'function': {'arguments': '{"a":3,"b":1000}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 62, 'total_tokens': 80, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_8bda4d3a2c', 'id': 'chatcmpl-CDTCpwYo7TSYSlhyUYLPZAX8HuMcC', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--4467a30d-acce-45e8-9bb5-55858ab7d724-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 1000}, 'id': 'call_oH5fpEySIA6sF44mywhcWINL', 'type': 'tool_call'}], usage_metada

In [25]:
llm_with_tools.invoke(messages).content

'The product of 3 and 1000 is 3000.'

In [42]:
# tool create
from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
  """
  This function fetches the currency conversion factor between a given base currency and a target currency
  """
  url = f'https://v6.exchangerate-api.com/v6/abff6d82d65d67425e3b988d/pair/{base_currency}/{target_currency}'

  response = requests.get(url)

  return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
  """
  given a currency conversion rate this function calculates the target currency value from a given base currency value
  """

  return base_currency_value * conversion_rate


In [43]:
convert.args

{'base_currency_value': {'title': 'Base Currency Value', 'type': 'integer'},
 'conversion_rate': {'title': 'Conversion Rate', 'type': 'number'}}

In [44]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1757289601,
 'time_last_update_utc': 'Mon, 08 Sep 2025 00:00:01 +0000',
 'time_next_update_unix': 1757376001,
 'time_next_update_utc': 'Tue, 09 Sep 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 88.2484}

In [45]:
convert.invoke({'base_currency_value':10, 'conversion_rate':85.16})

851.5999999999999

In [46]:
# tool binding
# llm = ChatOpenAI()

In [47]:
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [48]:
messages = [HumanMessage('What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd')]

In [49]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={})]

In [50]:
ai_message = llm_with_tools.invoke(messages)

In [51]:
messages.append(ai_message)

In [52]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'INR', 'target_currency': 'USD'},
  'id': 'call_vLTsCjsFSI2WS4xHoIqzko9I',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency_value': 10},
  'id': 'call_kCK46x3q0H1bKZeTS3fVEht7',
  'type': 'tool_call'}]

In [37]:
import json

for tool_call in ai_message.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)
    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    # append this tool message to messages list
    messages.append(tool_message1)
  # execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name'] == 'convert':
    # fetch the current arg
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)



In [38]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_WgZvl2JIkJxlV3l2Tb6ykfIT', 'function': {'arguments': '{"base_currency": "INR", "target_currency": "USD"}', 'name': 'get_conversion_factor'}, 'type': 'function'}, {'id': 'call_wuAJnsmFI2pOLe7LYO6zbQXF', 'function': {'arguments': '{"base_currency_value": 10}', 'name': 'convert'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 119, 'total_tokens': 172, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_8bda4d3a2c', 'id': 'chatcmpl-CDTCucb6InLTBo6YK5gbwtjYdCx7C', 'servi

In [39]:
llm_with_tools.invoke(messages).content

'The conversion factor from INR to USD is approximately 0.01133. Therefore, converting 10 INR to USD gives you approximately 0.1133 USD.'

In [40]:
from langchain.agents import initialize_agent, AgentType

# Step 5: Initialize the Agent ---
agent_executor = initialize_agent(
    tools=[get_conversion_factor, convert],
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,  # using ReAct pattern
    verbose=True  # shows internal thinking
)

C:\Users\Raghav Agarwal\AppData\Local\Temp\ipykernel_19388\627313024.py:4: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent_executor = initialize_agent(


In [41]:
# --- Step 6: Run the Agent ---
user_query = "Hi how are you?"

response = agent_executor.invoke({"input": user_query})



> Entering new AgentExecutor chain...
Thought: The user is greeting me and seems to be looking for a friendly interaction. There's no need for action here.
Action:
```
{
  "action": "Final Answer",
  "action_input": "I'm just a program, but I'm here to help you! How can I assist you today?"
}
```


> Finished chain.
